In [5]:
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.transforms import *
from pyspark.sql.functions import * 
from pyspark.sql.types import * 
from pyspark.sql import SparkSession
from awsglue.utils import *
from awsglue.dynamicframe import DynamicFrame
import sys  
from datetime import datetime 
import pandas as pd

In [6]:
sc = SparkContext()
glueContext=GlueContext(sc)
spark=glueContext.spark_session

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [227]:
df = spark.read.parquet("yellow_tripdata_2021-01.parquet",)

In [221]:
df.limit(1).toPandas() 

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,drop_off_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_duration


In [222]:
df.show(1)

[Stage 200:==========================================>              (3 + 1) / 4]

+---------+--------------------+---------------------+---------------+-------------+------------+------------------+------------------+--------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------+
|vendor_id|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|rate_code_id|store_and_fwd_flag|pickup_location_id|drop_off_location_id|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_duration|
+---------+--------------------+---------------------+---------------+-------------+------------+------------------+------------------+--------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------+
+---------+--------------------+---------------------+---------------+-------------+----------

In [229]:
rename_column_dict = {'VendorID':'vendor_id','RatecodeID':'rate_code_id','PULocationID':'pickup_location_id','DOLocationID':'drop_off_location_id'}

In [230]:
df = df.withColumnsRenamed(rename_column_dict)

In [249]:
cast_column_dict ={'vendor_id': col('vendor_id').cast(IntegerType()),
            'tpep_pickup_datetime':date_format(col('tpep_pickup_datetime'),'yyyy-MM-dd HH:mm:ss'),
           'tpep_dropoff_datetime':date_format(col('tpep_dropoff_datetime'),'yyyy-MM-dd HH:mm:ss'),
            'passenger_count':col('passenger_count').cast(IntegerType()),
            'trip_distance':col('trip_distance').cast(FloatType()),
            'rate_code_id':col('rate_code_id').cast(IntegerType()),
            'store_and_fwd_flag':when(col('store_and_fwd_flag').cast(StringType()) == 'Y', 1).otherwise(0),
            'pickup_location_id':col('pickup_location_id').cast(IntegerType()),
            'drop_off_location_id':col('drop_off_location_id').cast(IntegerType()),
            'payment_type':col('payment_type').cast(IntegerType()),
            'fare_amount':col('fare_amount').cast(FloatType()),
            'extra':col('extra').cast(FloatType()),
            'mta_tax':col('mta_tax').cast(FloatType()),
            'tip_amount':col('tip_amount').cast(FloatType()),
            'improvement_surcharge':col('improvement_surcharge').cast(FloatType()),
            'total_amount':col('total_amount').cast(DecimalType(10,2)),
            'congestion_surcharge':col('congestion_surcharge').cast(FloatType()),
            'airport_fee':coalesce(col('airport_fee').cast(FloatType()), lit(0.0)),
                  } 
df = df.withColumns(cast_column_dict)

In [250]:
df.limit(10).toPandas()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,drop_off_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,0,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,12.00,2.5,0.0
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,0.20,1,0,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.00,0.0,0.0
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,0,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,52.00,0.0,0.0
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,0,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.00,0.0,0.0
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,0,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.00,2.5,0.0
5,1,2021-01-01 00:16:29,2021-01-01 00:24:30,1,1.60,1,0,224,68,1,8.0,3.0,0.5,2.35,0.0,0.3,14.00,2.5,0.0
6,1,2021-01-01 00:00:28,2021-01-01 00:17:28,1,4.10,1,0,95,157,2,16.0,0.5,0.5,0.00,0.0,0.3,17.00,0.0,0.0
7,1,2021-01-01 00:12:29,2021-01-01 00:30:34,1,5.70,1,0,90,40,2,18.0,3.0,0.5,0.00,0.0,0.3,22.00,2.5,0.0
8,1,2021-01-01 00:39:16,2021-01-01 01:00:13,1,9.10,1,0,97,129,4,27.5,0.5,0.5,0.00,0.0,0.3,29.00,0.0,0.0
9,1,2021-01-01 00:26:12,2021-01-01 00:39:46,2,2.70,1,0,263,142,1,12.0,3.0,0.5,3.15,0.0,0.3,19.00,2.5,0.0


In [236]:
df.count()

1369769

In [217]:
df = df.filter('passenger_count <= 0 and passenger_count >= 6')

In [204]:
df = df.filter('trip_distance >= 5.0 and trip_distance <= 500.0')

In [205]:
df = df.filter('fare_amount > 0 ')

In [106]:
df = df.withColumn('Trip_ID', expr('uuid()'))

In [189]:
date_cols = {'tpep_pickup_date_id':date_format('tpep_pickup_datetime','yyyyMMdd'),'tpep_pickup_time_id':date_format('tpep_pickup_datetime','HHmmss'),
             'tpep_dropoff_date_id':date_format('tpep_dropoff_datetime','yyyyMMdd'),'tpep_dropoff_time_id':date_format('tpep_dropoff_datetime','HHmmss')
}

In [190]:
df = df.withColumns(date_cols)

In [206]:
df = df.withColumn('trip_duration',floor((unix_timestamp(df['tpep_dropoff_datetime']) - unix_timestamp(df['tpep_pickup_datetime']))/60))

In [209]:
df = df.filter(df['trip_duration'] < 1500)

In [216]:
df.sort(df.trip_duration.desc()).limit(10).toPandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_duration
0,2,2021-01-13 15:24:59,2021-01-14 15:24:52,1,22.15,5,0,132,265,1,118.0,0.0,0.0,15.00,19.87,0.3,153.169998,0.0,0.0,1439
1,2,2021-01-29 13:28:29,2021-01-30 13:27:38,1,7.77,1,0,226,138,2,31.5,0.0,0.5,0.00,0.00,0.3,32.299999,0.0,0.0,1439
2,2,2021-01-09 14:11:35,2021-01-10 14:09:43,1,7.83,1,0,163,33,1,24.5,0.0,0.5,6.95,0.00,0.3,34.750000,2.5,0.0,1438
3,2,2021-01-12 09:23:43,2021-01-13 09:22:32,1,8.43,1,0,238,261,1,27.5,0.0,0.5,6.16,0.00,0.3,36.959999,2.5,0.0,1438
4,2,2021-01-26 15:35:48,2021-01-27 15:34:46,1,5.14,1,0,231,170,2,17.0,0.0,0.5,0.00,0.00,0.3,20.299999,2.5,0.0,1438
5,2,2021-01-28 11:24:04,2021-01-29 11:22:06,1,5.59,1,0,116,237,1,26.0,0.0,0.5,5.86,0.00,0.3,35.160000,2.5,0.0,1438
6,2,2021-01-26 18:04:43,2021-01-27 18:02:15,1,8.92,1,0,162,181,1,28.0,1.0,0.5,6.46,0.00,0.3,38.759998,2.5,0.0,1437
7,2,2021-01-21 22:38:11,2021-01-22 22:36:09,3,10.71,1,0,142,92,2,35.5,0.5,0.5,0.00,0.00,0.3,39.299999,2.5,0.0,1437
8,2,2021-01-25 09:27:59,2021-01-26 09:25:31,1,5.86,1,0,170,33,2,19.5,0.0,0.5,0.00,0.00,0.3,22.799999,2.5,0.0,1437
9,2,2021-01-28 08:36:19,2021-01-29 08:33:48,1,5.26,1,0,238,244,1,18.0,0.0,0.5,4.26,0.00,0.3,25.559999,2.5,0.0,1437


In [109]:
df = df.drop('tpep_pickup_datetime','tpep_dropoff_datetime')

In [214]:
df.filter(df.passenger_count > 5).select(max(df.passenger_count)).show()

+--------------------+
|max(passenger_count)|
+--------------------+
|                   7|
+--------------------+

